# Strands Agent with Langfuse Observability on Amazon Bedrock AgentCore Runtime

## Overview

This notebook demonstrates deploying a Strands agent to Amazon Bedrock AgentCore Runtime with Langfuse observability integration. The implementation uses Amazon Bedrock Claude models and sends telemetry data to Langfuse through OpenTelemetry (OTEL).

## Key Components

- **Strands Agents**: Python framework for building LLM-powered agents with built-in telemetry support
- **Amazon Bedrock AgentCore Runtime**: Managed runtime service for hosting and scaling agents on AWS
- **Langfuse**: Open-source observability platform for LLM applications that receives traces via OTEL
- **OpenTelemetry**: Industry-standard protocol for collecting and exporting telemetry data

## Architecture

The agent is containerized and deployed to AgentCore Runtime, which provides HTTP endpoints for invocation. Telemetry data flows from the Strands agent through OTEL exporters to Langfuse for monitoring and debugging. The implementation disables AgentCore's default observability to use Langfuse instead.

## Prerequisites

- Python 3.10+
- AWS credentials configured with Bedrock and AgentCore permissions
- [Langfuse](https://langfuse.com/) account with API keys (public and secret keys)
- Docker installed locally
- Access to Amazon Bedrock Claude models in us-west-2

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Configure AWS Credentials

## Agent Implementation

The agent file (`strands_claude.py`) implements a travel agent with web search capabilities. Key configuration includes:
- Disabling AgentCore's default OTEL configuration
- Setting Langfuse endpoint and authentication
- Initializing Strands telemetry after environment variables are configured

In [ ]:
%%writefile strands_claude.py
import base64
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import os
from strands import Agent, tool
from strands.models import BedrockModel
from strands.telemetry import StrandsTelemetry
from ddgs import DDGS


# AWS Bedrock configuration
os.environ["BEDROCK_MODEL_ID"] = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Langfuse configuration
os.environ["OTEL_ENDPOINT"] = "https://us.cloud.langfuse.com" + "/api/public/otel"
os.environ["LANGFUSE_SECRET_API_KEY"] = "<Your-SECRET-API-KEY>"
os.environ["LANGFUSE_PUBLIC_API_KEY"] = "<Your-PUBLIC-API-KEY>"
os.environ["LANGFUSE_PROJECT_NAME"] = "<Your-Project-Name>"

@tool
def web_search(query: str) -> str:
    """
    Search the web for information using DuckDuckGo.
    
    Args:
        query: The search query
    
    Returns:
        A string containing the search results
    """
    try:
        ddgs = DDGS()
        results = ddgs.text(query, max_results=5)
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result.get('title', 'No title')}\n"
                f"   {result.get('body', 'No summary')}\n"
                f"   Source: {result.get('href', 'No URL')}\n"
            )
        
        return "\n".join(formatted_results) if formatted_results else "No results found."
        
    except Exception as e:
        return f"Error searching the web: {str(e)}"

# Function to initialize Bedrock model
def get_bedrock_model():
    model_id = os.getenv("BEDROCK_MODEL_ID", "anthropic.claude-3-haiku-20240307-v1:0")
    region = os.getenv("AWS_DEFAULT_REGION", "us-west-2")
    
    bedrock_model = BedrockModel(
        model_id=model_id,
        region_name=region,
        temperature=0.0,
        max_tokens=1024
    )
    return bedrock_model

# Initialize the Bedrock model
bedrock_model = get_bedrock_model()

# Define the agent's system prompt (exact from AWS sample)
system_prompt = """You are an experienced travel agent specializing in personalized travel recommendations 
with access to real-time web information. Your role is to find dream destinations matching user preferences 
using web search for current information. You should provide comprehensive recommendations with current 
information, brief descriptions, and practical travel details."""

app = BedrockAgentCoreApp()

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    os.environ["DISABLE_ADOT_OBSERVABILITY"] = "true"

    for k in [
        "OTEL_EXPORTER_OTLP_LOGS_HEADERS",
        "AGENT_OBSERVABILITY_ENABLED",
        "OTEL_PYTHON_DISTRO",
        "OTEL_RESOURCE_ATTRIBUTES",
        "OTEL_PYTHON_CONFIGURATOR",
        "OTEL_PYTHON_EXCLUDED_URLS"
        ]:
        os.environ.pop(k, None)

    ### Langfuse configuration``
    public_key = os.getenv("LANGFUSE_PUBLIC_API_KEY")
    secret_key = os.getenv("LANGFUSE_SECRET_API_KEY")

    # Create authentication token:
    auth_token = base64.b64encode(f"{public_key}:{secret_key}".encode()).decode()

    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.getenv("OTEL_ENDPOINT")
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {auth_token}"

    user_input = payload.get("prompt")
    print("User input:", user_input)

    # Initialize Strands telemetry and setup OTLP exporter
    strands_telemetry = StrandsTelemetry()
    strands_telemetry.setup_otlp_exporter()

    # Create the agent
    agent = Agent(
        model=bedrock_model,
        system_prompt=system_prompt,
        tools=[web_search]
    )

    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code. Please note that when using the `bedrock_agentcore_starter_toolkit` to configure your agent, it configures AgentCore Observability by default so, to use Braintrust, you need to remove configuration for AgentCore Observability as explained below:

<div style="text-align:left">
    <img src="../images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_langfuse_observability"
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

## Enable Langfuse Observability

To use Langfuse for observability, you need to modify the dockerfile before you deploy your agent to AgentCore Runtime.

**Important:** At the end of the Dockerfile, you'll find a CMD instruction that needs to be modified:

        * Before: `CMD ["opentelemetry-instrument", "python", "-m", "strands_langfuse_observability"]` 
        * After: `CMD ["python", "-m", "strands_langfuse_observability"]`

Remove the `"opentelemetry-instrument"` argument from the command. This change is crucial because it disables AgentCore's built-in observability and enables Langfuse observability instead.

## Deploy to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="../images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

## Check Deployment Status

Wait for the runtime to be ready before invoking:

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="../images/invoke.png" width=75%"/>
</div>

In [ ]:
from IPython.display import Markdown, display

invoke_response = agentcore_runtime.invoke({"prompt": "I'm planning a weekend trip to Tokyo. What are the must-visit places and local food I should try?"})
response_text = invoke_response['response'][0]
display(Markdown(response_text))

## View Traces in Langfuse

To view the traces:
1. Go to your Langfuse dashboard at https://cloud.langfuse.com
2. Navigate to your project
3. Click on "Traces" to view the telemetry data

The traces will include:
- Agent invocation details
- Tool calls (web search)
- Model interactions with latency and token usage
- Request/response payloads

## Cleanup (Optional)

Clean up the deployed resources:

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

## Summary

You have successfully deployed a Strands agent to Amazon Bedrock AgentCore Runtime with Langfuse observability. The implementation demonstrates:
- Integration of Strands agents with AgentCore Runtime
- Configuration of OpenTelemetry to send traces to Langfuse
- Proper initialization order to ensure telemetry configuration
- Invocation through both SDK and boto3 client

The agent is now running in a managed, scalable environment with full observability through Langfuse.